<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/use_cases/RAG/HelloLlamaCloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This demo app shows:
* How to run Llama 3 in the cloud hosted on Replicate
* How to use LangChain to ask Llama general questions and follow up questions
* How to use LangChain to load a recent web page - Hugging Face's [blog post on Llama 3](https://huggingface.co/blog/llama3) - and chat about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama 3 be able to answer questions about the data not publicly available when Llama 3 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

**Note** We will be using [Replicate](https://replicate.com/meta/meta-llama-3-8b-instruct) to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. You can also use other Llama 3 cloud providers such as [Groq](https://console.groq.com/), [Together](https://api.together.xyz/playground/language/meta-llama/Llama-3-8b-hf), or [Anyscale](https://app.endpoints.anyscale.com/playground) - see Section 2 of the Getting to Know Llama [notebook](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Getting_to_know_Llama.ipynb) for more information.

Let's start by installing the necessary packages:
- sentence-transformers for text embeddings
- FAISS gives us database capabilities
- LangChai provides necessary RAG tools for this demo

In [4]:
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu
!pip install bs4
!pip install replicate

In [5]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN


··········


Next we call the Llama 3 8b chat model from Replicate. You can also use Llama 3 70b model by replacing the `model` name with "meta/meta-llama-3-70b-instruct".

In [7]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.1 MB/s eta 0:00:00


In [8]:
from langchain_community.llms import Replicate
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens":500}
)

With the model set up, you are now ready to ask some questions. Here is an example of the simplest way to ask the model some general questions.

In [9]:
question = "who wrote the book Innovator's dilemma?"
answer = llm.invoke(question)
print(answer)

The book "The Innovator's Dilemma: When New Technologies Cause Great Firms to Fail" was written by Clayton M. Christensen, a Harvard Business School professor. The book was first published in 1997 and has since become a classic in the field of business and innovation.


We will then try to follow up the response with a question asking for more information on the book.

Since the chat history is not passed on Llama doesn't have the context and doesn't know this is more about the book thus it treats this as new query.


In [10]:
# chat history not passed so Llama doesn't have the context and doesn't know this is more about the book
followup = "tell me more"
followup_answer = llm.invoke(followup)
print(followup_answer)

I'd be happy to tell you more about myself.

I'm an AI assistant, which means I'm a computer program designed to assist and communicate with humans. My primary goal is to provide helpful and accurate information to answer your questions and complete tasks for you.

Here are some things I can do:

1. **Answer questions**: I can process natural language queries and provide relevant answers. Whether you're looking for general knowledge, definitions, or specific information, I'm here to help.
2. **Generate text**: I can create text based on a prompt or topic. This can be useful for writing articles, emails, or even entire books.
3. **Translate languages**: I can translate text from one language to another. I currently support translations in dozens of languages.
4. **Summarize content**: If you have a long piece of text and want to get a quick summary of the main points, I can help with that.
5. **Offer suggestions**: I can provide suggestions for things like gift ideas, travel destination

To get around this we will need to provide the model with history of the chat.

To do this, we will use  [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) to pass the chat history to the model and give it the capability to handle follow up questions.

In [11]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

Once this is set up, let us repeat the steps from before and ask the model a simple question.

Then we pass the question and answer back into the model for context along with the follow up question.

In [12]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

The book "The Innovator's Dilemma: When New Technologies Cause Great Firms to Fail" was written by Clayton Christensen, an American academic and business theorist. He's a professor at Harvard Business School and is known for his work on innovation, disruption, and strategy. The book was first published in 1997 and has since become a classic in the field of business and management. It explores the idea that established companies can struggle to innovate and adapt to new technologies, leading to their downfall.


In [13]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

I'd be happy to tell you more about the book and its author! Clayton Christensen's work is highly regarded in the business and academic communities, and "The Innovator's Dilemma" is considered one of his most influential books.

In the book, Christensen argues that established companies often struggle to innovate and adapt to new technologies because of their existing business models, organizational structures, and cultural norms. He uses the term "disruptive innovation" to describe the process by which new technologies or business models disrupt the status quo and create new markets or opportunities.

Christensen also introduces the concept of the "innovator's dilemma," which refers to the tension that exists between the need to innovate and the need to maintain profitability. He argues that established companies often prioritize short-term financial performance over long-term innovation, which can lead to their downfall.

The book is filled with examples of companies that have succes

Next, let's explore using Llama 3 to answer questions using documents for context.
This gives us the ability to update Llama 3's knowledge thus giving it better context without needing to finetune.

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
docs = loader.load()


We need to store our document in a vector store. There are more than 30 vector stores (DBs) supported by LangChain.
For this example we will use [FAISS](https://github.com/facebookresearch/faiss), a popular open source vector store by Facebook.
For other vector stores especially if you need to store a large amount of data - see [here](https://python.langchain.com/docs/integrations/vectorstores).

We will also import the HuggingFaceEmbeddings and RecursiveCharacterTextSplitter to assist in storing the documents.

In [15]:
# Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)

# Store the document into a vector store with a specific embedding model
vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database.

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

We then use `RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 3, thereby increasing its knowledge.

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to Llama to answer the question.

In [16]:
# use LangChain's RetrievalQA, to associate Llama 3 with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

question = "What's new with Llama 3?"
result = qa_chain({"query": question})
print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


According to the provided context, the new features and changes in Llama 3 compared to Llama 2 are:

* A new tokenizer that expands the vocabulary size to 128,256 (from 32K tokens in the previous version)
* The introduction of 4 new open LLM models based on the Llama 2 architecture, in two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions
* The ability to run on various types of consumer hardware and have a context length of 8K tokens
* A new version of Llama Guard, fine-tuned on Llama 3 8B and released as Llama Guard 2 (safety fine-tune)

These changes aim to improve the efficiency and multilingualism of the models, while also providing more deployment options and a larger vocabulary size.


Now, lets bring it all together by incorporating follow up questions.

First we ask a follow up questions without giving the model context of the previous conversation.
Without this context, the answer we get does not relate to our original question.

In [17]:
# no context passed so Llama 3 doesn't have enough context to answer so it lets its imagination go wild
result = qa_chain({"query": "Based on what architecture?"})
print(result['result'])

Based on the provided context, it appears that Llama 3 is based on the Meta Llama architecture.


As we did before, let us use the `ConversationalRetrievalChain` package to give the model context of our previous question so we can add follow up questions.

In [18]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [19]:
# let's ask the original question What's new with Llama 3?" again
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

According to the provided context, the new features and changes in Llama 3 compared to Llama 2 are:

* A new tokenizer that expands the vocabulary size to 128,256 (from 32K tokens in the previous version)
* The introduction of 4 new open LLM models based on the Llama 2 architecture, in two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions
* The ability to run on various types of consumer hardware and have a context length of 8K tokens
* A new version of Llama Guard, fine-tuned on Llama 3 8B and released as Llama Guard 2 (safety fine-tune)

These changes aim to improve the efficiency and multilingualism of the models, while also providing more deployment options and a larger vocabulary size.


In [20]:
# this time we pass chat history along with the follow up so good things should happen
chat_history = [(question, result["answer"])]
followup = "Based on what architecture?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

Based on the provided context, it appears that Llama 3 is based on the Meta Llama architecture.


In [21]:
# further follow ups can be made possible by updating chat_history like this:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "What changes in vocabulary size?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

According to the text, the vocabulary size has increased from 32K tokens in Llama 2 to 128,256 tokens in Llama 3.


**Note:** If results can get cut off, you can set "max_new_tokens" in the Replicate call above to a larger number (like shown below) to avoid the cut off.

```python
model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens": 1000}
```